<a href="https://colab.research.google.com/github/chaeminsoo/Ai_beginning/blob/master/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 신경망의 일반적인 학습 과정은 다음과 같습니다

# 학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의합니다.
# 데이터셋(dataset) 입력을 반복합니다.
# 입력을 신경망에서 전파(process)합니다.
# 손실(loss; 출력이 정답으로부터 얼마나 떨어져있는지)을 계산합니다.
# 변화도(gradient)를 신경망의 매개변수들에 역으로 전파합니다.
# 신경망의 가중치를 갱신합니다. 일반적으로 다음과 같은 간단한 규칙을 사용합니다: 새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()
      # 입력 이비지 채널 1개, 출력 채널 6개, 5x5의 정사각 컨볼루션 행렬
      # 컨볼루션 커널 정의
      self.conv1 = nn.Conv2d(1,6,5)
      self.conv2 = nn.Conv2d(6,16,5)
      # 아핀(affine) 연산: y = Wx+b
      self.fc1 = nn.Linear(16*5*5, 120)
      self.fc2 = nn.Linear(120, 84)
      self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
      # (2,2) 크기의 윈도우에 대해 max pooling
      x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
      # 크기가 제곱 수라면, 하나의 숫자만을 특정(specify)
      x = F.max_pool2d(F.relu(self.conv2(x)), 2)
      x = torch.flatten(x,1) # 배치 차원을 제외한 모든 차원을 하나로 평탄화
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
# forward 정의하면 backward 자동 정의 (beacause of autograd)

In [5]:
params = list(net.parameters()) # net.parameters(): 학습 가능한 매개 변수
print(len(params))
print(params[0].size()) # conv1의 .weight
# print(params)

10
torch.Size([6, 1, 5, 5])


In [6]:
input_ = torch.randn(1,1,32,32)
out = net(input_)
print(out)
print(out.shape)

tensor([[ 0.0785,  0.0665,  0.0440, -0.0258, -0.0168,  0.0251,  0.1245,  0.0790,
          0.0240, -0.0619]], grad_fn=<AddmmBackward0>)
torch.Size([1, 10])


In [7]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [8]:
# 현재까지 나온 것 요약

# torch.Tensor: backward() 같은 autograd 연산을 지원하는 다차원 배열 입니다. 또한 tensor에 대한 변화도를 갖고 있습니다.
# nn.Module: 신경망 모듈. 매개변수를 캡슐화(encapsulation)하는 간편한 방법 으로, GPU로 이동, 내보내기(exporting),
#            불러오기(loading) 등의 작업을 위한 헬퍼(helper)를 제공합니다.
# nn.Parameter: Tensor의 한 종류로, Module 에 속성으로 할당될 때 자동으로 매개변수로 등록 됩니다.
# autograd.Function: autograd 연산의 순방향과 역방향 정의 를 구현합니다.
#                    모든 Tensor 연산은 하나 이상의 Function 노드를 생성하며, 각 노드는 Tensor 를 생성하고 이력(history)을 인코딩 하는 함수들과 연결하고 있습니다.


# 지금까지 다룬 것:
# 신경망을 정의하는 것
# 입력을 처리하고 backward를 호출하는 것

# 앞으로의 내용:
# 손실을 계산
# 신경망 가중치를 갱신하는 것

In [9]:
# 손실 함수

In [10]:
output = net(input_)
target = torch.rand(10) # 임의의 정답
target = target.view(1,-1) # 출력과 같은 shape으로 만듦
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

tensor(0.2489, grad_fn=<MseLossBackward0>)


In [11]:
# input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
#       -> flatten -> linear -> relu -> linear -> relu -> linear
#       -> MSELoss
#       -> loss

In [11]:
# 역전파

In [13]:
net.zero_grad() # 모든 매개변수의 뱐화도 버퍼를 0으로 만듦

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0074,  0.0146,  0.0018,  0.0016,  0.0031, -0.0084])


In [ ]:
# 가중치 갱신

In [14]:
learning_rate = 0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

In [15]:
import torch.optim as optim

# Optimizer 생성
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정
optimizer.zero_grad() # 변화도 버퍼를 0으로
output = net(input_)
loss = criterion(output, target)
loss.backward()
optimizer.step() # update 진행